In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import itertools
import functions as fn 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from category_encoders import TargetEncoder

In [4]:
data = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/data_mining/assignment_2/code_personal/training_set_VU_DM.csv')
data_test = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/data_mining/assignment_2/code_personal/test_set_VU_DM.csv')

In [5]:
data

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,77700,3,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
4958343,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,88083,3,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
4958344,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,94508,3,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
4958345,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,128360,3,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,157.84,1


#### KNN classifier

-> would work a lot better with normalized data 

##### Dummy preprocessing

In [6]:
# comp vars aggregation 
df_knn = data.copy()
df_knn = fn.comp_aggregation(df_knn)
# adding relevance score 
df_knn["relevance"] = df_knn["booking_bool"].apply(lambda x: 5 if x == 1 else 0) + df_knn["click_bool"].apply(lambda x: 1 if x == 1 else 0) # does 5+1 btw
# drop datetime
df_knn = df_knn.drop(columns="date_time")
# replace all NaN with 0 -> temporary measure 
df_knn = df_knn.fillna(0)

# use only a subset of data 
df_small = df_knn.head(100000)

# split dataset - TODO: make sure not to split the query & property groups 
X = df_small.drop(columns="relevance")
X_train, X_test, y_train, y_test = train_test_split(X, df_small["relevance"], test_size=0.2, random_state=42)

# TODO: drop position 

##### Fitting model 

In [7]:
knn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree', metric='euclidean')
knn.fit(X_train, y_train)
# distances & indices of nearest neighbors (from train set) for each datapoint in the test set 
distances, indices = knn.kneighbors(X_test)  # around 12 minutes on full dataset 

##### Predicting relevance/ranking score

In [8]:
neighbor_values = []
# get relevance values of neighbors (from x_train) for x_test 
for neighbor_ids in indices:
    y_train_values = [ y_train.iloc[i] for i in neighbor_ids ]
    neighbor_values.append(y_train_values)
    
# get average relevance for each train instance 
relevance_average = np.mean(neighbor_values, axis=1)

In [9]:
# save property id & search id according to indices 
output_df = X_test[['srch_id', 'prop_id']].copy()
output_df['true_relevance'] = y_test
output_df['predicted_relevance'] = relevance_average

output_df

,srch_id,prop_id,true_relevance,predicted_relevance
75721,5069,85578,0,0.0
80184,5378,99219,0,0.0
19864,1345,76996,0,0.0
76699,5135,24242,0,0.0
92991,6258,63963,0,0.0
...,...,...,...,...
32595,2199,14107,0,0.0
29313,1969,118631,1,0.0
37862,2534,86606,0,0.0
53421,3527,52375,0,0.0


In [10]:
# np.savetxt('knn_test_output.txt', np.asarray(output_df))

##### calculating NDCG score based on ranking

how??? 

rank the grouped instances based on their rank? 
then rank the predicted grouped instances based on their ranks? 
then compute? 

--> think I should use the kaggle benchmarks thing 

Ok, do this -> use wikipedia formula, with relevance score in calculation (for true values) and ordering for predicted values ig. 

In [11]:
# TODO 

In [12]:
# rank the input according to their relevance scores? 

#### Little writeup 

With this knn we are getting the nearest neighbors (from the train set) for each datapoint in the test set. 
We then get all the relevance values of these neighbors
We then assign the average of those relevance values to each x in the test set. 
Then we create the ranking by grouping by search id and property id and ranking according to relevance score. 

Performance is currenlty horrendous, need to do a lot of pre-processing 

#### categorical feature encoding thing

categorical variables: 

(no mods to prop_id & search_id, although why not?? TODO)

- site_id -> ID of the point of sale (.com, .fr, ...)
- visitor_location_country_id -> ID of country where customer is located
- prop_country_id -> ID of country hotel is in
- prop_brand_bool -> 1: if the hotel is part of a major chain, 0: if not 
- promotion_flag -> 1: if there was a promotion, 0: if not 
- srch_destination_id -> ID of destination where hotel search was performed 
- srch_length_of_stay -> nb of nights in search query
- srch_booking_window -> nb of days between search & stay date 
- srch_adults_count -> nb of adults in search query 
- srch_children_count -> nb of children in search query 
- srch_room_count -> tf is even this 
- srch_saturday_night_bool -> 1: if includes saturday night, 0: if not 

In [13]:
dummy_data = fn.comp_aggregation(data.iloc[:10000].copy())
dummy_data["relevance"] = dummy_data["booking_bool"].apply(lambda x: 5 if x == 1 else 0) + df_knn["click_bool"].apply(lambda x: 1 if x == 1 else 0) # does 5+1 btw

In [14]:
dummy_data = fn.categorical_encoding(dummy_data)
dummy_data

,srch_id,date_time,visitor_hist_starrating,visitor_hist_adr_usd,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,visitor_location_country_id,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,relevance
0,1,2013-04-04 08:32:15,NaN,NaN,893,3,3.5,1,2.83,0.0438,...,0.214286,0.155733,0.214286,0.208236,0.208487,0.188748,0.179428,0.180537,0.195467,0.0
1,1,2013-04-04 08:32:15,NaN,NaN,10404,4,4.0,1,2.20,0.0149,...,0.214286,0.155733,0.214286,0.208236,0.208487,0.188748,0.179428,0.180537,0.195467,0.0
2,1,2013-04-04 08:32:15,NaN,NaN,21315,3,4.5,1,2.20,0.0245,...,0.214286,0.155733,0.214286,0.208236,0.208487,0.188748,0.179428,0.180537,0.195467,0.0
3,1,2013-04-04 08:32:15,NaN,NaN,27348,2,4.0,1,2.83,0.0125,...,0.214286,0.155733,0.214286,0.208236,0.208487,0.188748,0.179428,0.180537,0.195467,0.0
4,1,2013-04-04 08:32:15,NaN,NaN,29604,4,3.5,1,2.64,0.1241,...,0.214286,0.155733,0.214286,0.208236,0.208487,0.188748,0.179428,0.180537,0.195467,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,703,2013-06-20 11:13:33,NaN,NaN,78064,3,4.5,1,2.08,0.1175,...,0.177634,0.155733,0.240000,0.208236,0.243243,0.181041,0.179428,0.180537,0.195467,0.0
9996,703,2013-06-20 11:13:33,NaN,NaN,79720,3,3.5,1,1.95,0.0423,...,0.177634,0.155733,0.240000,0.208236,0.243243,0.181041,0.179428,0.180537,0.195467,0.0
9997,703,2013-06-20 11:13:33,NaN,NaN,94363,3,4.0,1,1.39,NaN,...,0.177634,0.155733,0.240000,0.208236,0.243243,0.181041,0.179428,0.180537,0.195467,0.0
9998,703,2013-06-20 11:13:33,NaN,NaN,96276,2,NaN,0,0.69,NaN,...,0.177634,0.155733,0.240000,0.208236,0.243243,0.181041,0.179428,0.180537,0.195467,0.0


In [15]:
# add prop_id & search_id??
categorical_features = ['site_id', 'prop_country_id', 'visitor_location_country_id', 
                        'promotion_flag', 'srch_destination_id', 'srch_length_of_stay', 
                        'srch_booking_window', 'srch_adults_count', 'srch_children_count',
                        'srch_room_count', 'srch_saturday_night_bool', 'relevance'] 

# assumes that relevance target is already there 

# changing datatype 
# get all categorical features 
X = dummy_data[categorical_features]

alpha = 1

X.to_csv('test.csv')

# let's just code it by hand :)
encoded_features = X.copy()

# overall target average 
overall_avg = X['relevance'].mean()

for var in X.drop('relevance', axis=1):
    # get all unique values 
    unique_values = X[var].unique()

    for value in unique_values:
        # get avg feature value per categorical value 
        matching_target_values = X.loc[X[var] == value, 'relevance']
        target_avg = matching_target_values.mean()
        # weighted average 
        weighted_avg = (alpha * target_avg + (1-alpha) * overall_avg) 
        # update encoded feature 
        encoded_features[var].replace(value, weighted_avg, inplace=True)

# replace df with encoded features 
dummy_data = dummy_data.drop(categorical_features, axis=1)
dummy_data[categorical_features] = encoded_features

dummy_data


# overall target average 
# overall_avg = X['relevance'].mean()


# weighted average of features specific & overall average 


# # fitting encoder 
# encoder = TargetEncoder()
# encoder.fit(X, y)

# X_encoded = encoder.transform(X)

# X_encoded.to_csv('test.csv')

# # TODO: check if this worked right 


,srch_id,date_time,visitor_hist_starrating,visitor_hist_adr_usd,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,visitor_location_country_id,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,relevance
0,1,2013-04-04 08:32:15,NaN,NaN,893,3,3.5,1,2.83,0.0438,...,0.214286,0.155733,0.214286,0.208236,0.208487,0.188748,0.179428,0.180537,0.195467,0.0
1,1,2013-04-04 08:32:15,NaN,NaN,10404,4,4.0,1,2.20,0.0149,...,0.214286,0.155733,0.214286,0.208236,0.208487,0.188748,0.179428,0.180537,0.195467,0.0
2,1,2013-04-04 08:32:15,NaN,NaN,21315,3,4.5,1,2.20,0.0245,...,0.214286,0.155733,0.214286,0.208236,0.208487,0.188748,0.179428,0.180537,0.195467,0.0
3,1,2013-04-04 08:32:15,NaN,NaN,27348,2,4.0,1,2.83,0.0125,...,0.214286,0.155733,0.214286,0.208236,0.208487,0.188748,0.179428,0.180537,0.195467,0.0
4,1,2013-04-04 08:32:15,NaN,NaN,29604,4,3.5,1,2.64,0.1241,...,0.214286,0.155733,0.214286,0.208236,0.208487,0.188748,0.179428,0.180537,0.195467,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,703,2013-06-20 11:13:33,NaN,NaN,78064,3,4.5,1,2.08,0.1175,...,0.177634,0.155733,0.240000,0.208236,0.243243,0.181041,0.179428,0.180537,0.195467,0.0
9996,703,2013-06-20 11:13:33,NaN,NaN,79720,3,3.5,1,1.95,0.0423,...,0.177634,0.155733,0.240000,0.208236,0.243243,0.181041,0.179428,0.180537,0.195467,0.0
9997,703,2013-06-20 11:13:33,NaN,NaN,94363,3,4.0,1,1.39,NaN,...,0.177634,0.155733,0.240000,0.208236,0.243243,0.181041,0.179428,0.180537,0.195467,0.0
9998,703,2013-06-20 11:13:33,NaN,NaN,96276,2,NaN,0,0.69,NaN,...,0.177634,0.155733,0.240000,0.208236,0.243243,0.181041,0.179428,0.180537,0.195467,0.0


idea is to convert categorical features into numerical featues 

NOTES: crucial for KNN!!
       LambdaMART is primarily designed to work with numerical features 
       -> google more advantages (& disadvantages)

think he does -> weighted average of target encoding (ie. average value for that feature) & overall average value of the target 
-> probably an overfitting tradeoff mechanism 

workflow:

- find out which features are categorical 
- make list of these features 
- for each category, get the avearge value of the target 
- get the overall average value of the target 
- do weighted average of both for each row 